In [ ]:
import random
import numpy as np
import math
import os

In [ ]:
#Limites de la funcion
limSup = 90
limInf = 40

In [ ]:
#calcular genotipo
def iniciarPoblacion(cantIndividuos, cantCromosomas):
    poblacion = np.zeros((cantIndividuos, cantCromosomas))
    for individuo in range(cantIndividuos):
        for cromosoma in range(cantCromosomas):
            poblacion[individuo][cromosoma] = random.randint(0,1)
    return poblacion

In [ ]:
#calcular fenotipo de toda la poblacion, recibe el genotipo binario y devuelve el fenotipo sin nomalizar
def calcularFenotipo(cantIndividuos, cantCromosomas, poblacion):
    fenotipo = np.zeros(cantIndividuos)
    for individuo in range(cantIndividuos):
        sumatoria = 0
        for cromosoma in range(cantCromosomas-1, -1, -1):
            if poblacion[individuo][cromosoma] == 1.0:
                sumatoria += math.pow(2, (cantCromosomas-1)-cromosoma)
        fenotipo[individuo] = sumatoria
    return fenotipo

In [ ]:
#Calcula el Fenotipo de un individuo o elemento de la matriz de la poblacion aqui lo regresa normalizado (poblacion)
def calcularFenotipoElemento(cantIndividuos, cantCromosomas, poblacion):
    fenotipo = 0
    sumatoria = 0 
    for cromosoma in range(cantCromosomas-1, -1, -1):
        if poblacion[cromosoma] == 1.0:
            sumatoria += math.pow(2, (cantCromosomas-1)-cromosoma)
    fenotipo = sumatoria
    fenotipo = ((((limSup-limInf)* fenotipo)/( math.pow(2, cantCromosomas)-1)))+limInf
    return fenotipo

In [ ]:
#Normalizamos el fenotipo, recibe un vectir de fenotipos de toda la poblacion (individuo)
def normalizarFenotipo(limSup, limInf, cantIndividuos, fenotipo):
    fenotipoNormalizado = np.zeros(cantIndividuos)
    # Normalizar los fenotipos
    for individuo in range(cantIndividuos):
        fenotipoNormalizado[individuo] = (((limSup-limInf)* fenotipo[individuo])/( math.pow(2, cantCromosomas)-1))+limInf
    return fenotipoNormalizado

In [ ]:
#Funcion a minimizar con el algoritmo genetico
U = lambda t:((204165.5)/(330-(2*t))+(10400/(t-20)))

In [ ]:
def convertFitness(fi):
    return (1/(1+fi))

In [ ]:
def probFi(Fi, suma):
    return Fi/suma

In [ ]:
#Calculo del fitness
def calcularFitnes(fenotipoNormalizado, cantIndividuos):
    fitness = np.zeros(cantIndividuos)
    for individuo in range(cantIndividuos):
        fitness[individuo] = U(fenotipoNormalizado[individuo])
    return fitness 

In [ ]:
#Calculo del fitness
def calcularFitnes1(fenotipoNormalizado, cantIndividuos):
    fitness = np.zeros(cantIndividuos)
    cf = np.zeros(cantIndividuos)
    probF = np.zeros(cantIndividuos)
    for individuo in range(cantIndividuos):
        fitness[individuo] = U(fenotipoNormalizado[individuo])
        cf[individuo] = convertFitness(fitness[individuo])
    for individuo in range(cantIndividuos):
        probF[individuo] = probFi(cf[individuo], np.sum(cf))
    return fitness, cf, probF 

In [ ]:
#Método 1 Seleccion por ruleta
def seleccionPorRuleta(p_x, fitness, cantIndividuos, cantCromosomas, poblacion):
    fitnessSleccion = np.zeros(cantIndividuos)
    individuosPadres = np.empty((0,  cantCromosomas), float)
    #######Ruleta
    ruleta = []
    ruleta.append(0.0)
    for p in range(1,len(p_x)+1):
        sumAcc = 0
        for P in range(1,p+1):
            sumAcc += p_x[P-1]
        ruleta.append(sumAcc)
    sets_individuos = []
    #print(ruleta)
    for padreSeleccion in range(cantIndividuos):
        set_x = np.random.uniform(low=0, high=1, size=1)
        #print(set_x)
        for i in range(0,len(ruleta)):
            if(set_x>=ruleta[i] and set_x<ruleta[i+1]):
                sets_individuos.append(i)
    for i in range(len(sets_individuos)):
        temp=sets_individuos[i]
        fitnessSleccion[i]= fitness[temp]
        individuosPadres=np.append(individuosPadres, np.array([poblacion[temp]]), axis=0)
    return fitnessSleccion, individuosPadres

In [ ]:
#Método 2 Seleccion por rank
def seleccionPorRank(fitness, cantIndividuos, cantCromosomas, poblacion):
    fitness.sort()
    #print(fitness)
    leng=len(fitness)
    #print(leng)
    leng=leng*(leng+1)/2
    #print(leng)
    prob=[]
    for i in range(len(fitness)):
        prob.append((i+1)/leng)
    fitnessSleccion, individuosPadres=seleccionPorRuleta(prob,fitness,cantIndividuos,cantCromosomas,poblacion)
    return fitnessSleccion, individuosPadres

In [ ]:
#Método 3 Seleccion por competicion
def seleccionPorCombate(fitness, cantIndividuos, cantCromosomas, poblacion):
    #Se inicializa el vector que contendra los fitness mejores de los competidores
    fitnessSleccion = np.zeros(cantIndividuos)
    #Se inicializa una matriz que contendra los individuos ganadores de la competición
    individuosPadres = np.empty((0,  cantCromosomas), float)
    #Ahora inicializamos un for para la cantidad de individiduos que contiene la poblacion para la competecion
    for padreSeleccion in range(cantIndividuos):
        #Obtener el fitness del individuo seleccionado
        individuo = fitness[padreSeleccion]
        #Se selecciona un competidor al azar de la poblacion que no sea así mismo la competencia
        x = list(range(cantIndividuos))
        x.pop(padreSeleccion)
        posCompetidor = random.choice(x)
        competidor = fitness[posCompetidor]
        #Aqui se compara los fitness de individuo de la poblacion contra el competidor seleccionado
        #aleatoriamente, si el individuo tiene el de menor fitness es el que gana
        if individuo < competidor:
            #Agregamos el fitness del individuo al vector del fitness seleccionado
            fitnessSleccion[padreSeleccion]= individuo
            #Y agregamos el genotipo individuo a la matriz de individuos seleccionados
            individuosPadres = np.append(individuosPadres, np.array([poblacion[padreSeleccion]]), axis=0)
        else:
            #Caso contrario si el competidor tiene el menor fitness lo agregamos al vector de mejor fitness
            fitnessSleccion[padreSeleccion]= competidor 
            #Y agregamos el genotipo competidor a la matriz de individuos seleccionados
            individuosPadres = np.append(individuosPadres, np.array( [poblacion[posCompetidor]]), axis=0)
    return fitnessSleccion, individuosPadres

In [ ]:
#Crossover y mutación
#esta funcion recibe como paramentros los genotipos y fitness que fueron seleccionados en algunos de los 3 metodos
def crossover(fitnessSleccion, individuosPadres):
    #se genera una matriz para almacenar los hijos de los padres
    individuosHijos = np.empty((0,  cantCromosomas), float)
    #Se genera un punto de division de la cadena de bits de cada individuo
    #este punto de division se genera de la cantidad de cromosomas que conforma al individuo
    puntoDivision = random.randint(1,cantCromosomas-2)
    #Se inicializa el hijo
    hijo = 0
    p = math.pow(cantCromosomas, -1)
    print('Probabilidad de mutacion:'+str(p))
    #Para la cantidad de individuos de la población se hara un for para realizar el crossover
    for nuevoIndividuo in range(cantIndividuos):
        #Se obtiene aletoriamanente un indice para elegir el inidivio1 con el que va cruzar
        individuo1 = random.randint(0,cantIndividuos-1)
        #Se saca el inidivio de la matriz para evitar el cruce entre el mismo individuo
        x = list( range(cantIndividuos))
        x.pop(individuo1)
        #Se obtiene aletoriamanente un indice para elegir el inidivio2 con el que va cruzar
        individuo2 = random.choice(x)
        ind1 = individuosPadres[individuo1]
        ind2 = individuosPadres[individuo2]
        #Aqui se toma los bits menor significativos para el inidividuo 1 a partir del punto de division
        x = ind1[0:puntoDivision]
        #Aqui se toma los bits mas significativos para el inidividuo 2 a partir del punto de division
        y = ind2[puntoDivision:cantCromosomas]
        #Se concantenan los cromosomas por parte del individuo 1 y del individuo 2
        hijo = np.concatenate((x, y))
        #Y se agrega a a la matriz el nuevo hijo generado
        individuosHijos = np.append(individuosHijos, [hijo], axis=0)
    #Se realiza la mutacion
    cont = 0
    #Para cada nuevo individuo generado el cruce se realiza una posible mutacion en un cromosoma del individuo
    for nuevoIndividuo in range(cantIndividuos):
        for cromomosoma in range(cantCromosomas):
            #Se obtiene un probabilidad aleatioria de tipo uniforme para saber si el bit mutara o no
            prob = random.uniform(0,1)
            if prob <= p:
                individuosHijos[nuevoIndividuo][cromomosoma] = int (not individuosHijos[nuevoIndividuo][cromomosoma])
    #Se regresan los nuevos hijos
    return individuosHijos

In [ ]:
#seleccionar el menor fitness despues de hacer la mutacion
#cada vez que se tiene una poblacion se debe obtener el mejor individuo
def seleccionarMejorIndividuo(poblacion, fitness):
    MejorFitness = fitness[0]
    posicion = 0
    genotipo = 0
    for i in range(cantIndividuos):
        if MejorFitness >= fitness[i]:
            MejorFitness = fitness[i]
            posicion = i
    genotipo = poblacion[posicion]
    return posicion, MejorFitness, genotipo

In [ ]:
# e = 0.001
while(True):
    metodo = int(input("¿Cuál método deseas utilizar?\n 1. Torneo\n 2. Ruleta\n 3. Rank\n "))
    imprimir = int(input("¿Desea imprimir todo?\n1.-Si\n2.-No\n"))
    cantIndividuos = int(input("Cantidad de individuos:"))
    cantCromosomas = int(input("Cantidad de cromosomas:"))
    generaciones = int(input("Cantidad de generaciones:"))
    poblacion = iniciarPoblacion(cantIndividuos, cantCromosomas)
    fenotipo = calcularFenotipo(cantIndividuos, cantCromosomas, poblacion)
    fenotipoNormalizado = normalizarFenotipo(limSup, limInf, cantIndividuos, fenotipo)
    if(imprimir==1):
        print("-------------Inicialización-------------------")
        print("\nPoblacion inicial")
        print(poblacion)
        print("\nFenotipo")
        print(fenotipo.T)
        print("\nFenotipo Normalizado")
        print(fenotipoNormalizado.T)
        #fitness = calcularFitnes(fenotipoNormalizado, cantIndividuos)
    if(metodo==1):
        fitness = calcularFitnes(fenotipoNormalizado, cantIndividuos)
    if(metodo==2):
        fitness, cf, probF = calcularFitnes1(fenotipoNormalizado, cantIndividuos)
    if(metodo==3):    
        fitness= calcularFitnes(fenotipoNormalizado, cantIndividuos)
    if(imprimir==1):
        print("\nFitness")
        print(fitness)
    position, mejorFitness, genotipo = seleccionarMejorIndividuo(poblacion, fitness)
    print('****************Mejor valor****************************************')
    print("Iteración: " + str(0))
    print("Posición mejor fitness y inidividuo: " + str(position))
    print("Fitness: " + str(mejorFitness))
    print("Genotipo: " + str(genotipo))
    f = calcularFenotipoElemento(cantIndividuos, cantCromosomas, genotipo)
    mejorIndividuoAnterior = mejorFitness
    print("Fenotipo Normalizado")
    print(f)
    print('*******************************************************************')
    for generacion in range(generaciones):
        #print("###################################################################")
            # continuar cruzando
        if(metodo==1):
            fitnessSleccion, individuosPadres = seleccionPorCombate(fitness, cantIndividuos, cantCromosomas, poblacion)
        if(metodo==2):
            fitnessSleccion, individuosPadres =seleccionPorRuleta(probF, fitness, cantIndividuos, cantCromosomas, poblacion)
        if(metodo==3):     
            fitnessSleccion, individuosPadres =seleccionPorRank(fitness, cantIndividuos, cantCromosomas, poblacion)
        poblacion = crossover(fitnessSleccion, individuosPadres)
        fenotipo = calcularFenotipo(cantIndividuos, cantCromosomas, poblacion)
        fenotipoNormalizado = normalizarFenotipo(limSup, limInf, cantIndividuos, fenotipo)
        
        if(imprimir==1):
            print("\nFitness selección")
            print(fitnessSleccion)
            print("\nIndividuos selección")
            print(individuosPadres)
            print("\nFenotipo")
            print(fenotipo.T)
            print("\nFenotipo Normalizado")
            print(fenotipoNormalizado.T)
        fitness = calcularFitnes(fenotipoNormalizado, cantIndividuos)
        print("\nFitness")
        print(fitness)
        position, mejorFitness, genotipo = seleccionarMejorIndividuo(poblacion, fitness)
        print('****************Mejor valor****************************************')
        print("Iteración: " + str(generacion))
        print("Posición: " + str(position))
        print("Fitness: " + str(mejorFitness))
        print("Genotipo: " + str(genotipo))
        f = calcularFenotipoElemento(cantIndividuos, cantCromosomas, genotipo)
        print("f normalizado")
        print(f)
        print("hijos:")
        print(poblacion)
        print('*******************************************************************')
        #if (abs(mejorIndividuoAnterior- mejorFitness) < e): 
        #    break
        mejorIndividuoAnterior = mejorFitness
        print("###################################################################")
    continuar = int(input('Desea Continuar?\n1.-Sí\n2.-No\n'))
    if(continuar==1): 
        imprimir=0
        os.system ("cls") 
        pass
    elif(continuar==2): 
        break